In [4]:
!python -m venv .venv
!source .venv/bin/activate

!pip install --upgrade pip wheel setuptools==59.5.0

!pip install --upgrade datasets pandas wandb simpletransformers torch

  Using cached pip-22.0.2-py3-none-any.whl (2.1 MB)
  Using cached setuptools-60.7.1-py3-none-any.whl (1.0 MB)
  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tensorflow-io-gcs-filesystem>=0.23.1, which is not installed.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.21.5 which is incompatible.


In [5]:
# Import silicone dataset
from datasets import load_dataset
maptask = load_dataset("silicone", "maptask")

def cluster_speaker(batch):
    speakers = batch["Speaker"]
    utterances = batch["Utterance"]
    acts = batch["Label"]
    result = []
    j = 0
    num = len(speakers)
    
    while j < num:
        target_spkr = speakers[j]
        opp_spkr = 'g' if target_spkr == 'f' else 'f'
        try:
            k = speakers.index(opp_spkr, j)

        except Exception as e:
            k = num
        
        result.append([utterances[j:k], acts[j:k]])
        j = k
        
    return result

def process_data(dataset, split):
    data = []
    data_slice = dataset[split][:]
    clustered_set = cluster_speaker(data_slice)
    for i in range(len(clustered_set)):
        for j in range(len(clustered_set[i][0])):
            if i+j == 0:
                data.append(["", clustered_set[i][0][j], clustered_set[i][1][j]])
                continue
            if j == 0:
                data.append([clustered_set[i-1][0][-1], clustered_set[i][0][j], clustered_set[i][1][j]])
                continue

            data.append([". ".join(clustered_set[i][0][:j]), clustered_set[i][0][j], clustered_set[i][1][j]])
    
    return data

import pandas as pd

train_data = process_data(maptask, "train")
train_df = pd.DataFrame(train_data)
train_df.columns = ["text_a", "text_b", "labels"]

eval_data = process_data(maptask, "validation")
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]

Reusing dataset silicone (/home/jupyter/.cache/huggingface/datasets/silicone/maptask/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Set up wandb
import wandb

key = "2ee46b01a610d015968a7abe18a895014b237fbc"
project = "maptask"
entity = "diwank"

wandb.login(key=key)
wandb.init(project=project, entity=entity)

# Turn off debug logs
import transformers
transformers.logging.set_verbosity_error()


# model configuration
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

model_args = ClassificationArgs(
    n_gpu=2,
    num_train_epochs=64, 
    learning_rate=1e-5,
    model_type="deberta", 
    model_name="microsoft/deberta-base",
    # wandb_project=project,
    use_multiprocessing=False,
    train_batch_size=128,
    evaluate_during_training=True,
    # use_early_stopping=False,
    # early_stopping_patience=12,
    scheduler="polynomial_decay_schedule_with_warmup",
)

# Create a ClassificationModel
model = ClassificationModel("deberta", "microsoft/deberta-base", args=model_args, num_labels=12)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
# Train model
model.train_model(train_df, eval_df=eval_df)

/home/jupyter/.local/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/128 [00:00<?, ?it/s]

Running Epoch 0 of 128:   0%|          | 0/164 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/2963 [00:00<?, ?it/s]

...

Running Epoch 69 of 128:   0%|          | 0/164 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/2963 [00:00<?, ?it/s]

Running Epoch 70 of 128:   0%|          | 0/164 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
!gsutil cp -r ./outputs gs://internal.whitehead.ai/maptask/`date '+%m-%d-%Y_%H_%M_%S'`/outputs

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)